In [1]:
# ==========================================
# 1. CLEAN START & INSTALL
# ==========================================
import os
import sys
import gc
import torch

# Aggressive cleanup before start
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    gc.collect()

# Install libs offline
LIB_PATH = "offline_libs"
ZIP_PATH = "/kaggle/input/zno-libs-final/offline_libs.zip"

if not os.path.exists(LIB_PATH):
    if os.path.exists(ZIP_PATH):
        print("📦 Unzipping libraries...")
        !unzip -q {ZIP_PATH} -d .
    elif os.path.exists("/kaggle/input/zno-libs-final/offline_libs"):
        LIB_PATH = "/kaggle/input/zno-libs-final/offline_libs"

print(f"📦 Installing from {LIB_PATH}...")
!pip install --no-index --find-links={LIB_PATH} bitsandbytes peft accelerate transformers tokenizers safetensors sentencepiece > /dev/null
print("✅ Installation Complete!")

# ==========================================
# 2. CONFIGURATION (STABILITY MODE)
# ==========================================
BATCH_SIZE = 1           # <--- Найбезпечніший варіант. 0% ризику OOM.
MAX_CONTEXT_LEN = 1100   # <--- Зменшили з 1500. Цього достатньо для тексту + прикладів.
BASE_MODEL_PATH = "/kaggle/input/qwen2.5/transformers/7b-instruct/1" 
ADAPTER_PATH = "/kaggle/input/zno-my-adapter"
TEST_FILE_PATH = "/kaggle/input/zno-data/zno.test.jsonl" 

# ==========================================
# 3. LOAD MODEL
# ==========================================
import pandas as pd
import json
import numpy as np
from tqdm import tqdm
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel

if not torch.cuda.is_available():
    raise SystemError("❌ RUNNING ON CPU. Turn on GPU!")

print(f"⏳ Loading Model...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    local_files_only=True 
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_PATH, trust_remote_code=True, local_files_only=True)
tokenizer.padding_side = "left" 
tokenizer.pad_token = tokenizer.eos_token 

print(f"🔗 Attaching Adapter...")
model = PeftModel.from_pretrained(base_model, ADAPTER_PATH, local_files_only=True)
model.eval()

# ==========================================
# 4. TOKEN MAPPING (Cyrillic Candidates)
# ==========================================
candidates = ["А", "Б", "В", "Г", "Д"]
candidate_ids = []

for c in candidates:
    ids = tokenizer.encode(c, add_special_tokens=False)
    candidate_ids.append(ids[-1])

print(f"🎯 Candidate IDs: {candidate_ids}")
answer_map = {0: "А", 1: "Б", 2: "В", 3: "Г", 4: "Д"}

# ==========================================
# 5. PROMPT (PAPER OPTIMIZED)
# ==========================================
EXAMPLES = """
Питання: Слово з орфографічною помилкою є в рядку
Варіанти:
А: контрастний, баластний, форпостний
Б: пестуни, хвастливий, кістлявий
В: студентський, диригентський, туристський
Г: тижневий, серцевий, злісний
Д: учасник, яхтсмен, стільниковий
Відповідь: А

Питання: Укажіть рядок, у якому всі слова пишуться з великої літери
Варіанти:
А: (Ш,ш)евченківські вірші, (К,к)иївські вулиці
Б: (Д,д)ніпровські хвилі, (Л,л)ьвівська кава
В: (П,п)івденний (Б,б)уг, (З,з)олоті (В,в)орота
Г: (Н,н)аціональний (Б,б)анк, (В,в)ерховна (Р,р)ада
Д: (М,м)іністерство (О,о)світи, (К,к)абмін
Відповідь: В
"""

def create_prompt(item):
    q = item.get('question', '')
    if 'answers' in item:
        opts = "\n".join([f"{opt['marker']}: {opt['text']}" for opt in item['answers']])
    else:
        opts = str(item.get('answers', ''))
    
    instruction = "Дай відповідь буквою-варіантом відповіді з наданих варіантів."
    return f"<|im_start|>user\n{instruction}\n\nПриклади:\n{EXAMPLES}\n\nПитання: {q}\nВаріанти:\n{opts}<|im_end|>\n<|im_start|>assistant\nВідповідь:"

# ==========================================
# 6. INFERENCE LOOP (STABLE LOGITS)
# ==========================================
if not os.path.exists(TEST_FILE_PATH):
    print("⚠️ Path not found, searching...")
    for root, dirs, files in os.walk("/kaggle/input"):
        if "zno.test.jsonl" in files:
            TEST_FILE_PATH = os.path.join(root, "zno.test.jsonl")
            print(f"✅ Found at: {TEST_FILE_PATH}")

test_data = []
with open(TEST_FILE_PATH, "r", encoding="utf-8") as f:
    for line in f:
        try: test_data.append(json.loads(line))
        except: pass

print(f"🚀 Starting STABLE RUN (Batch 1) on {len(test_data)} items...")
results = []

for i in tqdm(range(0, len(test_data), BATCH_SIZE)):
    # Aggressive Memory Cleanup
    if i % 50 == 0: 
        torch.cuda.empty_cache()
        
    batch_items = test_data[i : i + BATCH_SIZE]
    batch_prompts = [create_prompt(item) for item in batch_items]
    batch_ids = [item.get('id') for item in batch_items]

    inputs = tokenizer(
        batch_prompts, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=MAX_CONTEXT_LEN # Reduced length
    ).to("cuda")

    with torch.inference_mode(): 
        outputs = model(**inputs)
        next_token_logits = outputs.logits[:, -1, :]
        candidate_scores = next_token_logits[:, candidate_ids]
        best_indices = torch.argmax(candidate_scores, dim=1).cpu().numpy()

    for q_id, idx in zip(batch_ids, best_indices):
        results.append({"id": q_id, "answer": answer_map[idx]})

    # Save frequently
    if i % 10 == 0:
        pd.DataFrame(results).to_csv("submission.csv", index=False)

# Final Save
pd.DataFrame(results).to_csv("submission.csv", index=False)
print(f"✅ DONE! Saved to submission.csv")

📦 Installing from /kaggle/input/zno-libs-final/offline_libs...
✅ Installation Complete!


2026-01-13 19:16:25.972798: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1768331786.171742      24 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1768331786.230049      24 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1768331786.706998      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768331786.707045      24 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1768331786.707047      24 computation_placer.cc:177] computation placer alr

⏳ Loading Model...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

🔗 Attaching Adapter...


/usr/local/lib/python3.12/dist-packages/peft/config.py:165: UserWarning: Unexpected keyword arguments ['alora_invocation_tokens', 'arrow_config', 'ensure_weight_tying', 'peft_version', 'use_bdlora'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


🎯 Candidate IDs: [36155, 60332, 16206, 37114, 24110]
⚠️ Path not found, searching...
✅ Found at: /kaggle/input/gen-ai-ucu-2025-task-3/zno.test.jsonl
🚀 Starting STABLE RUN (Batch 1) on 751 items...


100%|██████████| 751/751 [20:30<00:00,  1.64s/it]

✅ DONE! Saved to submission.csv
